# Notebook de ánalisis de información

In [1]:
# librerias base
import math
import os
import shutil 
import sys

#librerias externas
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp  

# librerias propias
# Carpeta del proyecto
dir_proyecto, _ = os.path.split(os.getcwd())
sys.path.append(dir_proyecto)
from SCRIPTS.funciones import *

In [2]:
# Esta instrucción define el directorio local
df = leer_parquet(os.path.join(dir_proyecto,'INPUTS'))
df.head(2)

In [3]:
print("\033[1mDimensiones de la data\n\033[0m")

print("Las dimensiones de la data son: " +str(df.shape[0]) +" filas y " + str(df.shape[1])+ " columnas\n")

print("\033[1mTipos de variables\n\033[0m")

print(df.info())

Se cuenta con información de 39.884 contratos y cada uno con 71 variables descriptivas del mismo. 


Al cargar la data, se observa existen variables con una cantidad significativa de valores nulos, por lo tanto se realiza el siguiente tratamiento: 

### 1. Tratamiento de Valores null

In [4]:
variables_con_nulos = df.isna().sum()

# Filtrar variables con valores perdidos
variables_con_nulos = variables_con_nulos[variables_con_nulos > 0]

for variable, cantidad_nulos in variables_con_nulos.items():
    print(f"{variable}, {cantidad_nulos}")
    
print("\nSe tiene un total de " + str(df.isna().sum().sum()) + " datos pérdidos")

Teniendo en cuenta se tiene un total de 130.743 valores nulos, se realizará el siguiente tratamiento:
    
1. Se procederá a eliminar las variables relacionadas con fechas. Esto debido a que, además de tener una gran cantidad de valores nulos, consideramos que no tendrán un impacto significativo en el modelo propuesto para el estudio.
    
2. Así mismo se eliminaran las filas de las demás variables que representan menos del 1%.

In [5]:
df = df.drop(['fecha_inicio_liquidacion','fecha_fin_liquidacion','fecha_de_inicio_de_ejecucion','fecha_de_fin_de_ejecucion','fecha_de_inicio_del_contrato'], axis=1)
df = df.dropna()

### 2. Selección inicial de variables.

Después de una exhaustiva revisión del diccionario de datos disponibles en el conjunto de contratos electrónicos del SECOP II, que puede ser consultado en el enlace: https://www.datos.gov.co/Gastos-Gubernamentales/SECOP-II-Contratos-Electr-nicos/jbjy-vk9h, hemos determinado que algunas variables no serán consideradas en nuestro modelo de estudio debido a su falta de relevancia. Las variables que excluiremos son las siguientes: 

* 'Localización' y 'ciudad' son lo mismo por lo tanto dejamos ciudad. 
* Información detallada del proveedor elegido no será tomada en cuenta en nuestro análisis, ya que el princial objetivo es generar una clasificación por sectores que permita a los proveedores presentar sus postulaciones.
* También excluiremos identificadores únicos dentro de cada contrato, como 'id_contrato', 'referencia_del_contrato', entre otros ya que no aportan información significativa para la clasificación. 

In [6]:
df_final = df.drop(['localizaci_n','proceso_de_compra', 'id_contrato', 'referencia_del_contrato',
                    'justificacion_modalidad_de','tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
                    'es_grupo', 'es_pyme','estado_bpin', 'c_digo_bpin', 'anno_bpin','espostconflicto', 'urlproceso',
                    'dias_adicionados','nombre_representante_legal','nacionalidad_representante_legal',
                    'tipo_de_identificaci_n_representante_legal','identificaci_n_representante_legal', 
                    'g_nero_representante_legal','ultima_actualizacion','codigo_entidad', 'codigo_proveedor',
                    'fecha_de_firma', 'fecha_de_fin_del_contrato'], axis=1)

In [7]:
df_final.shape

Se contará con 40 variables para ánalisis descriptivo. 

### 3. Tipos de variables:

Se realizó el cambio de tipo de variables, teniendo en cuenta que esta considerando todas de tipo object, como se muestra a continuación: 

In [8]:
df_final.info()

In [9]:
df_final


In [10]:

df_final['liquidaci_n'] = df['liquidaci_n'].apply(lambda x: 1 if x== "Si" else 0)
df_final['habilita_pago_adelantado'] = df['habilita_pago_adelantado'].apply(lambda x: 1 if x== "Si" else 0)

In [11]:
# Variables continuas: 

Variables_num = [
    'valor_del_contrato',
    'valor_de_pago_adelantado',
    'valor_facturado',
    'valor_pendiente_de_pago',
    'valor_pagado',
    'valor_amortizado',
    'valor_pendiente_de',
    'valor_pendiente_de_ejecucion',
    'saldo_cdp',
    'saldo_vigencia','presupuesto_general_de_la_nacion_pgn','recursos_de_credito','recursos_propios',
    'sistema_general_de_regal_as','recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_']
for i in Variables_num:
    df_final[i] = df_final[i].astype(int)

### 4. Ánalisis descriptivo de variables continuas

In [12]:
def calcular_estadisticas(df:pd.DataFrame)->pd.DataFrame:
    estadisticas = pd.DataFrame((df.describe(percentiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    estadisticas = estadisticas.T
    estadisticas['range'] = estadisticas['max']- estadisticas['min']
    estadisticas['median'] = [np.median(df.loc[:,i]) for i in estadisticas.index]
    estadisticas['skew'] = [sp.skew(df.loc[:,i]) for i in estadisticas.index]
    estadisticas['kurtosis'] = [sp.kurtosis(df.loc[:,i]) for i in estadisticas.index]
    estadisticas = estadisticas[['count', 'mean', 'median', 'std', 'min','50%', '60%', '90%', 'max']].round(2)
    return estadisticas



print("\n**Estadísticas de resumen de las variables continuas\n")
display(calcular_estadisticas(df_final[Variables_num]))

In [13]:
df_final

# Análisis de variables

In [14]:
df = df_final.iloc[:,:-1]

In [15]:
df.head(3)

In [16]:
df['orden'].unique()

In [17]:
df['destino_gasto'].unique()

In [18]:
df['origen_de_los_recursos'].unique()

In [19]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.violinplot(df['valor_del_contrato'])
plt.show()

In [20]:
# Gráfico de densidad
sns.kdeplot(df['valor_del_contrato'])

In [21]:
valor = df['valor_del_contrato'].sort_values(ascending = False)

In [22]:
valor.iloc[0:60]

# Variables categóricas

In [23]:
columnas_categoricas = df_final.select_dtypes(include=['object'])
columnas_categoricas

In [24]:
columnas_categoricas.columns

In [25]:
columnas_categoricas['sector'].unique()

In [26]:
sector = columnas_categoricas.groupby(by=['sector']).count()
#sector_ordenado = sector.sort_values(ascending=False)
sector

In [27]:
columnas_categoricas['sector'].unique()

In [28]:
import session_info

session_info.show(html=False)